In [1]:
import pandas as pd
import numpy as np

# 將NAV日報酬率幾何平均成月報酬率

In [2]:
df = pd.read_csv('/Users/Kang/Desktop/HW2/CSVs/5_1 NAV.csv', index_col = 'Date')
df = df.set_index(pd.to_datetime(df.index))
df = df[['NAV', 'NAV_CP', 'NAV_RI']]
df

NAV        NAV_CP        NAV_RI
Date                                                
2009-12-31  10000.000000  10000.000000  10000.000000
2010-01-04  10162.236852  10000.000000  10000.000000
2010-01-05  10187.654451  10000.000000  10000.000000
2010-01-06  10208.897786  10000.000000  10000.000000
2010-01-07  10198.266008  10000.000000  10000.000000
2010-01-08  10233.769026  10000.000000  10000.000000
2010-01-11  10263.187962  10000.000000  10000.000000
2010-01-12  10193.819622  10000.000000  10000.000000
2010-01-13  10243.288323  10000.000000  10000.000000
2010-01-14  10275.766855  10000.000000  10000.000000
2010-01-15  10202.235858  10000.000000  10000.000000
2010-01-19  10294.106951  10000.000000  10000.000000
2010-01-20  10160.348549  10000.000000  10000.000000
2010-01-21   9989.936057  10000.000000  10000.000000
2010-01-22   9826.968131  10000.000000  10000.000000
2010-01-25   9886.645233  10000.000000  10000.000000
2010-01-26   9807.492888  10000.000000  10000.000000
2010-01-27   9821.145077  10000.000000  10000.000000
2010-01-28   9746.656369  10000.000000  10000.000000
2010-01-29   9683.348154  10000.000000  10000.000000
2010-02-01   9795.434809  10115.720512  10116.787303
2010-02-02   9853.446775  10174.053940  10176.989610
2010-02-03   9811.509072  10131.797929  10133.639986
2010-02-04   9555.066701   9865.573653   9866.918371
2010-02-05   9545.671492   9856.033154   9857.498785
2010-02-08   9471.224584   9778.443923   9779.801990
2010-02-09   9595.825185   9907.841384   9909.325848
2010-02-10   9570.933938   9882.705169   9884.085555
2010-02-11   9651.790989   9967.158140   9968.463559
2010-02-12   9626.251847   9940.860584   9941.886785
...                  ...           ...           ...
2017-11-16  18992.277204  19579.336353  19644.974939
2017-11-17  18986.354204  19574.189032  19637.918388
2017-11-20  19004.110622  19592.443433  19656.649048
2017-11-21  19132.124277  19722.078673  19788.995965
2017-11-22  19160.830610  19751.967004  19818.466251
2017-11-24  19192.256707  19783.730422  19851.662390
2017-11-27  19094.873481  19685.208620  19750.981081
2017-11-28  19169.249411  19760.329179  19828.141706
2017-11-29  19084.997900  19674.322365  19742.086071
2017-11-30  19097.639495  19686.855150  19755.960001
2017-12-01  19077.280363  19664.160211  19733.688427
2017-12-04  19050.784990  19637.740893  19705.968912
2017-12-05  19014.693935  19599.568378  19667.919174
2017-12-06  18935.806562  19517.735771  19585.337563
2017-12-07  18958.584799  19542.161197  19610.002000
2017-12-08  19042.961264  19629.771710  19697.474116
2017-12-11  19086.787152  19675.258854  19743.205021
2017-12-12  19080.348381  19667.366813  19736.274077
2017-12-13  19147.403670  19735.441988  19804.613740
2017-12-14  19098.088396  19683.757469  19752.912038
2017-12-15  19159.083655  19746.807138  19816.112935
2017-12-18  19254.019890  19845.522141  19915.118919
2017-12-19  19134.879660  19724.458886  19792.592605
2017-12-20  19103.701622  19693.356984  19761.662612
2017-12-21  19136.727710  19727.867070  19795.130220
2017-12-22  19199.264038  19792.148820  19859.736942
2017-12-26  19186.628039  19778.867131  19845.804928
2017-12-27  19236.065462  19828.448125  19895.972114
2017-12-28  19292.377857  19886.636699  19954.639224
2017-12-29  19313.548932  19908.437250  19976.470956

[2014 rows x 3 columns]

In [3]:
#換算成daily_return
daily_return = df.pct_change()
daily_return = daily_return.dropna(how = 'all') #drop掉2009-12-31

In [4]:
#所有ETF名稱
all_ticker = daily_return.columns.tolist()

#挑出所有年月的最後一天
all_max_day_m = []
for i in range(daily_return.index.year.min(), daily_return.index.year.max()+1):
    for j in range(1, 13):
        mday = daily_return.loc[(daily_return.index.year == i) & (daily_return.index.month == j)].index.day.max()
        all_max_day_m.append(str(i) + '-' + ('0'+str(j) if j < 10 else str(j)) + '-' + str(mday))

all_max_day_m = [i for i in all_max_day_m if i[8:] != 'nan']

In [5]:
#建立monthly_return用來儲存月報酬
monthly_return = pd.DataFrame(index = all_max_day_m, columns = all_ticker)
monthly_return = monthly_return.set_index(pd.DatetimeIndex(monthly_return.index))

In [6]:
#換算成monthly return
for i in monthly_return.index:
    for j in all_ticker:
        d_return = daily_return[j][(daily_return.index.year == i.year) & (daily_return.index.month == i.month)]
        monthly_return[j][i] = (1 + d_return).cumprod()[-1] - 1

# 將LIBOR算術平均成月利率

In [7]:
#LIBOR利率，可用來當rf，但2008~2017的資料筆數卻比較多？要用要看怎麼merge
LIBOR = pd.read_csv('/Users/Kang/Desktop/HW2/CSVs/0_3 LIBOR_3M.csv', index_col = 'DATE')
LIBOR = LIBOR[LIBOR['USD3MTD156N'] != '.']
LIBOR = LIBOR.astype({'USD3MTD156N' : float})
LIBOR = LIBOR.div(100)
LIBOR = LIBOR.set_index(pd.to_datetime(LIBOR.index))

In [8]:
#建立LIBOR_m用來儲存月libor，並增加一個column
LIBOR_m = pd.DataFrame(index = all_max_day_m)
LIBOR_m['Libor monthly mean'] = np.nan
LIBOR_m = LIBOR_m.set_index(pd.DatetimeIndex(LIBOR_m.index))

In [9]:
#直接將一個月之Libor取平均，再除12，就是月均Libor
for i in LIBOR_m.index:
    d_return = LIBOR[(LIBOR.index.year == i.year) & (LIBOR.index.month == i.month)]
    LIBOR_m['Libor monthly mean'][i] = d_return.mean()
    
LIBOR_m = LIBOR_m / 12

# 結果一：直接使用兩個dataframe分別計算

In [10]:
print('Sharpe Ratios:')
(monthly_return.mean().values - LIBOR_m.mean().values) / monthly_return.std()
#(monthly_return.mean().values - LIBOR_m.mean().values) / monthly_return.std().values

Sharpe Ratios:


NAV       0.241041
NAV_CP    0.252825
NAV_RI    0.255849
dtype: float64

# 結果二：合併兩個dataframe的寫法

In [11]:
merge = pd.merge(monthly_return, LIBOR_m, on = monthly_return.index)
merge = merge.rename(index=str, columns={"key_0": "Date"})
merge = merge.set_index('Date')
merge = merge.set_index(pd.DatetimeIndex(merge.index))

In [12]:
print('Sharpe Ratios:')
(merge[['NAV', 'NAV_CP', 'NAV_RI']].mean() - merge['Libor monthly mean'].mean()) / merge[['NAV', 'NAV_CP', 'NAV_RI']].std()

Sharpe Ratios:


NAV       0.241041
NAV_CP    0.252825
NAV_RI    0.255849
dtype: float64

In [13]:
merge

NAV       NAV_CP       NAV_RI  Libor monthly mean
Date                                                                 
2010-01-29   -0.0316652            0            0            0.000208
2010-02-26    0.0158793    0.0157109     0.016035            0.000209
2010-03-31    0.0525646     0.052383    0.0527272            0.000224
2010-04-30    0.0103947    0.0109714    0.0105974            0.000260
2010-05-28   -0.0645278   -0.0643039   -0.0650973            0.000382
2010-06-30   -0.0146184   -0.0148859   -0.0146242            0.000447
2010-07-30     0.074716     0.075021    0.0738036            0.000425
2010-08-31   -0.0141455   -0.0137488   -0.0142809            0.000302
2010-09-30    0.0658189    0.0668989    0.0659986            0.000243
2010-10-29    0.0254036    0.0257106    0.0255231            0.000241
2010-11-30   -0.0157816   -0.0152419   -0.0157043            0.000239
2010-12-31     0.054333    0.0540303    0.0548369            0.000252
2011-01-31   0.00684051   0.00674981   0.00708978            0.000253
2011-02-28    0.0160955    0.0158394    0.0161903            0.000260
2011-03-31  -0.00133584  -0.00131759  -0.00136304            0.000257
2011-04-29    0.0397061    0.0401256    0.0393517            0.000234
2011-05-31  -0.00690361  -0.00718637   -0.0067412            0.000217
2011-06-30     -0.01476     -0.01499   -0.0148383            0.000207
2011-07-29  -0.00245564  -0.00263342  -0.00255034            0.000208
2011-08-31   -0.0468026   -0.0481121   -0.0472746            0.000244
2011-09-30   -0.0789491   -0.0802434   -0.0781937            0.000292
2011-10-31    0.0866677    0.0873981    0.0872535            0.000339
2011-11-30   -0.0168015   -0.0167833   -0.0168205            0.000396
2011-12-30   0.00198902    0.0016717   0.00205763            0.000463
2012-01-31    0.0533781    0.0535952    0.0528241            0.000472
2012-02-29    0.0315636    0.0317563    0.0316611            0.000419
2012-03-30   0.00508439   0.00519188    0.0054377            0.000394
2012-04-30  -0.00473344  -0.00488884  -0.00475172            0.000389
2012-05-31   -0.0526066   -0.0533385   -0.0531228            0.000389
2012-06-29    0.0396869    0.0400956    0.0398757            0.000388
...                 ...          ...          ...                 ...
2015-07-31   0.00286165   0.00287368   0.00267938            0.000242
2015-08-31   -0.0549758   -0.0555686   -0.0545039            0.000267
2015-09-30   -0.0148859   -0.0149035   -0.0150509            0.000276
2015-10-30    0.0506396    0.0512039    0.0500754            0.000268
2015-11-30  -0.00763512  -0.00768131  -0.00762717            0.000309
2015-12-31   -0.0155202   -0.0155509   -0.0157343            0.000444
2016-01-29   -0.0316288   -0.0319334   -0.0317744            0.000516
2016-02-29  0.000537256  0.000590823  0.000767677            0.000519
2016-03-31    0.0647026    0.0652344    0.0639154            0.000527
2016-04-29 -0.000179441 -0.000450409 -3.87135e-05            0.000527
2016-05-31   0.00660035   0.00645187   0.00665014            0.000538
2016-06-30    0.0227456      0.02329    0.0225514            0.000543
2016-07-29    0.0368475    0.0365283    0.0369507            0.000580
2016-08-31 -0.000972934 -0.000878051  -0.00109989            0.000675
2016-09-30   0.00770487     0.007769   0.00795283            0.000708
2016-10-31   -0.0177843   -0.0181259   -0.0177681            0.000732
2016-11-30    -0.015002   -0.0151606   -0.0148375            0.000757
2016-12-30    0.0123992    0.0125782    0.0124172            0.000813
2017-01-31    0.0275826    0.0279858    0.0278626            0.000855
2017-02-28    0.0231805    0.0231927    0.0231263            0.000870
2017-03-31    0.0094425   0.00922669   0.00973515            0.000946
2017-04-28    0.0137033    0.0134407    0.0137844            0.000966
2017-05-31    0.0148682    0.0148226    0.0150425            0.000988
2017-06-30    0.0100874    0.0100788    0.0101742            0.001052
2017-07-31    0.0200777    0.0

In [14]:
#merge.to_csv('/Users/Kang/Desktop/6_1 Monthly Returns.csv')